[![deep-learning-notes](https://github.com/semilleroCV/deep-learning-notes/raw/main/assets/banner-notebook.png)](https://github.com/semilleroCV/deep-learning-notes)

## Inception V1

In [1]:
%%capture
#@title **Install required packages**

! pip install torchmetrics

In [2]:
#@title **Importing libraries**

import torch # 2.3.1+cu121
import torchinfo #1.8.0

import torch.nn as nn

In [3]:
# Note: Not all dependencies have the __version__ method.

print(f"torch version: {torch.__version__}")
print(f"torchinfo version: {torchinfo.__version__}")

torch version: 2.3.1+cu121
torchinfo version: 1.8.0


#### Inception V1 architecture code

In [4]:
class ConvBlock(nn.Module):
  def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int):
    super(ConvBlock, self).__init__()

    self.conv_blk = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

  def forward(self, x):
    x = self.conv_blk(x)
    return x


class InceptionBlock(nn.Module):
  def __init__(self, in_channels: int, output_1x1: int, reduction_3x3: int, output_3x3: int,
               reduction_5x5: int, output_5x5: int, output_pooling: int):
    super(InceptionBlock, self).__init__()

    "output_1x1, reduction_3x3, output_3x3 , reduction_5x5 , output_5x5 , out_pooling"

    self.branch_1 = ConvBlock(in_channels, output_1x1, kernel_size=1, stride=1, padding=0)

    self.branch_2 = nn.Sequential()
    self.branch_2.add_module("b2_conv1", ConvBlock(in_channels, reduction_3x3, kernel_size=1, stride=1, padding=0))
    self.branch_2.add_module("b2_conv2", ConvBlock(reduction_3x3, output_3x3, kernel_size=3, stride=1, padding=1))

    self.branch_3 = nn.Sequential()
    self.branch_3.add_module("b3_conv1", ConvBlock(in_channels, reduction_5x5, kernel_size=1, stride=1, padding=0))
    self.branch_3.add_module("b3_conv2", ConvBlock(reduction_5x5, output_5x5, kernel_size=5, stride=1, padding=2))

    self.branch_4 = nn.Sequential()
    self.branch_4.add_module("b4_pool", nn.MaxPool2d(kernel_size=3, stride=1, padding=1))
    self.branch_4.add_module("b4_conv1", ConvBlock(in_channels, output_pooling, kernel_size=1, stride=1, padding=0))

  def forward(self, x):
    branch_1 = self.branch_1(x)
    branch_2 = self.branch_2(x)
    branch_3 = self.branch_3(x)
    branch_4 = self.branch_4(x)

    return torch.cat([branch_1, branch_2, branch_3, branch_4], dim=1)


class InceptionV1(nn.Module):
  def __init__(self, in_channels, num_classes: int):
    super(InceptionV1, self).__init__()

    self.Inceptionv1 = nn.Sequential()

    """conv1, maxpool1"""
    self.Inceptionv1.add_module("conv1", ConvBlock(in_channels, 64, kernel_size=7, stride=2, padding=3)) 
    self.Inceptionv1.add_module("pool1", nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    """conv2, maxpool2"""
    self.Inceptionv1.add_module("conv2", ConvBlock(64, 64, kernel_size=1, stride=1, padding=0))
    self.Inceptionv1.add_module("conv3", ConvBlock(64, 192, kernel_size=3, stride=1, padding=1)) 
    self.Inceptionv1.add_module("pool2", nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    """inception 3a, inception 3b and maxpool"""
    self.Inceptionv1.add_module("Incept_3a", InceptionBlock(192, 64, 96, 128, 16, 32, 32))
    self.Inceptionv1.add_module("Incept_3b", InceptionBlock(256, 128, 128, 192, 32, 96, 64))
    self.Inceptionv1.add_module("pool3", nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    """inception 4a, 4b, 4c, 4d, 4e and maxpool"""

    self.Inceptionv1.add_module("Incept_4a", InceptionBlock(480, 192, 96, 208, 16, 48, 64))
    self.Inceptionv1.add_module("Incept_4b", InceptionBlock(512, 160, 112, 224, 24, 64, 64))
    self.Inceptionv1.add_module("Incept_4c", InceptionBlock(512, 128, 128, 256, 24, 64, 64))
    self.Inceptionv1.add_module("Incept_4d", InceptionBlock(512, 112, 144, 288, 32, 64, 64))
    self.Inceptionv1.add_module("Incept_4e", InceptionBlock(528, 256, 160, 320, 32, 128, 128))
    self.Inceptionv1.add_module("pool4", nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

    """inception 5a, 5b and avgpool"""

    self.Inceptionv1.add_module("Incept_5a", InceptionBlock(832, 256, 160, 320, 32, 128, 128))
    self.Inceptionv1.add_module("Incept_5b", InceptionBlock(832, 384, 192, 384, 48, 128, 128))
    self.Inceptionv1.add_module("pool5", nn.AdaptiveAvgPool2d((1, 1)))

    """ droput, flatten and linear (fc)"""
    self.Inceptionv1.add_module("dropout", nn.Dropout(p=0.4))
    self.Inceptionv1.add_module("flatten", nn.Flatten())
    self.Inceptionv1.add_module("fc", nn.Linear(1024, num_classes))

  def forward(self, x):
    x = self.Inceptionv1(x)
    return x

In [5]:
model = InceptionV1(in_channels=3, num_classes=1000)
torchinfo.summary(model, (3, 224, 224), batch_dim = 0)

Layer (type:depth-idx)                             Output Shape              Param #
InceptionV1                                        [1, 1000]                 --
├─Sequential: 1-1                                  [1, 1000]                 --
│    └─ConvBlock: 2-1                              [1, 64, 112, 112]         --
│    │    └─Sequential: 3-1                        [1, 64, 112, 112]         9,536
│    └─MaxPool2d: 2-2                              [1, 64, 56, 56]           --
│    └─ConvBlock: 2-3                              [1, 64, 56, 56]           --
│    │    └─Sequential: 3-2                        [1, 64, 56, 56]           4,224
│    └─ConvBlock: 2-4                              [1, 192, 56, 56]          --
│    │    └─Sequential: 3-3                        [1, 192, 56, 56]          110,976
│    └─MaxPool2d: 2-5                              [1, 192, 28, 28]          --
│    └─InceptionBlock: 2-6                         [1, 256, 28, 28]          --
│    │    └─ConvBlock: 3